In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv('/kaggle/input/cirrhosis-patient-survival-prediction/cirrhosis.csv')
data2 = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')

In [3]:
cols1 = [x for x in data1.columns]
cols1

['ID',
 'N_Days',
 'Status',
 'Drug',
 'Age',
 'Sex',
 'Ascites',
 'Hepatomegaly',
 'Spiders',
 'Edema',
 'Bilirubin',
 'Cholesterol',
 'Albumin',
 'Copper',
 'Alk_Phos',
 'SGOT',
 'Tryglicerides',
 'Platelets',
 'Prothrombin',
 'Stage']

In [4]:
cols2 = [y for y in data2.columns]

In [5]:
un = [x for x in zip(cols1,cols2)]

In [6]:
un

[('ID', 'id'),
 ('N_Days', 'N_Days'),
 ('Status', 'Drug'),
 ('Drug', 'Age'),
 ('Age', 'Sex'),
 ('Sex', 'Ascites'),
 ('Ascites', 'Hepatomegaly'),
 ('Hepatomegaly', 'Spiders'),
 ('Spiders', 'Edema'),
 ('Edema', 'Bilirubin'),
 ('Bilirubin', 'Cholesterol'),
 ('Cholesterol', 'Albumin'),
 ('Albumin', 'Copper'),
 ('Copper', 'Alk_Phos'),
 ('Alk_Phos', 'SGOT'),
 ('SGOT', 'Tryglicerides'),
 ('Tryglicerides', 'Platelets'),
 ('Platelets', 'Prothrombin'),
 ('Prothrombin', 'Stage'),
 ('Stage', 'Status')]

In [7]:
data = pd.concat([data1,data2],axis=0)

In [8]:
data.isnull().sum()

ID               7905
N_Days              0
Status              0
Drug              106
Age                 0
Sex                 0
Ascites           106
Hepatomegaly      106
Spiders           106
Edema               0
Bilirubin           0
Cholesterol       134
Albumin             0
Copper            108
Alk_Phos          106
SGOT              106
Tryglicerides     136
Platelets          11
Prothrombin         2
Stage               6
id                418
dtype: int64

In [9]:
data['id']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
7900    7900.0
7901    7901.0
7902    7902.0
7903    7903.0
7904    7904.0
Name: id, Length: 8323, dtype: float64

In [10]:
cols = [x for x in data1.columns]
for x in cols:
    print(x)

ID
N_Days
Status
Drug
Age
Sex
Ascites
Hepatomegaly
Spiders
Edema
Bilirubin
Cholesterol
Albumin
Copper
Alk_Phos
SGOT
Tryglicerides
Platelets
Prothrombin
Stage


In [11]:
data = data.drop('ID',axis = 1)
data = data.drop('id',axis = 1)
cols.remove('ID')
#cols.remove('id')

In [12]:
objectCols = []
for i in cols: 
    if data[f'{i}'].dtype == object :
        objectCols.append(i)

In [13]:
objectCols.remove('Status')

In [14]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
encoder = OrdinalEncoder()
labelencoder = LabelEncoder()

In [15]:
encodedData = data.copy()
encodedData[objectCols] = encoder.fit_transform(data[objectCols])
encodedData['Status'] = labelencoder.fit_transform(data['Status'])

In [16]:
encodedData['Status'].value_counts()

Status
0    5197
2    2826
1     300
Name: count, dtype: int64

In [17]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()

In [18]:
imputedData = pd.DataFrame(imputer.fit_transform(encodedData),columns = cols)

In [19]:
imputedData.isnull().sum()

N_Days           0
Status           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
dtype: int64

In [20]:
#smoten = SMOTEN()

In [21]:
X = imputedData[cols]
y = imputedData['Status']

In [22]:
X = X.drop('Status',axis = 1)

In [23]:
X.shape

(8323, 18)

In [24]:
y.value_counts()

Status
0.0    5197
2.0    2826
1.0     300
Name: count, dtype: int64

In [25]:
yn = keras.utils.to_categorical(y)

In [26]:
yn = pd.DataFrame(yn,columns = ['Status_C','Status_CL','Status_D']).to_numpy()
xn = X.to_numpy()

In [27]:
batch =  int(xn.shape[0]/28)

In [28]:
model = keras.Sequential()

#model.add(keras.layers.Flatten()

model.add(keras.layers.Dense(batch*2,activation = 'relu'))
model.add(keras.layers.GaussianDropout(0.2))

model.add(keras.layers.Dense(int(batch),activation = 'relu'))
model.add(keras.layers.GaussianDropout(0.2))


model.add(keras.layers.Dense(3,activation = 'sigmoid'))

In [29]:
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.01),loss = keras.losses.CategoricalCrossentropy(),metrics = [keras.metrics.CategoricalAccuracy(),keras.metrics.CategoricalCrossentropy()])

In [30]:
callback = keras.callbacks.EarlyStopping(
    monitor='val_categical_crossentropy',
    min_delta=0.01,
    patience=20,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

In [31]:
history = model.fit(xn,yn,batch_size = batch,epochs = 100,callbacks = callback,validation_split = 0.25,verbose = 0)

In [32]:
test = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')
ID = test['id']
test = test.drop('id',axis = 1)
test[objectCols] = encoder.transform(test[objectCols])

In [33]:
pred = model.predict(test)

165/165 [==============================] - 0s 1ms/step


In [34]:
predicted = pd.DataFrame(pred)
predicted

,0,1,2
0,0.755808,0.151373,0.641182
1,0.755808,0.151373,0.641182
2,0.755808,0.151373,0.641182
3,0.755808,0.151373,0.641182
4,0.755808,0.151373,0.641182
...,...,...,...
5266,0.755808,0.151373,0.641182
5267,0.755808,0.151373,0.641182
5268,0.755808,0.151373,0.641182
5269,0.755808,0.151373,0.641182


In [35]:
sub = pd.DataFrame({'id':ID,'Status_C':predicted[0],'Status_CL':predicted[1],'Status_D':predicted[2]})

In [36]:
sub.to_csv("submission.csv", index = False)